# Les 12: Labo - Oplossingen

**Mathematical Foundations - IT & Artificial Intelligence**

---

Voorbeeldoplossingen voor de capstone projecten.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)

In [ ]:
# Complete Neural Network Library
class Layer(ABC):
    def __init__(self):
        self.params, self.grads, self.training = {}, {}, True
    @abstractmethod
    def forward(self, x): pass
    @abstractmethod
    def backward(self, dout): pass
    def __call__(self, x): return self.forward(x)

class Loss(ABC):
    @abstractmethod
    def forward(self, y_pred, y_true): pass
    @abstractmethod
    def backward(self): pass
    def __call__(self, y_pred, y_true): return self.forward(y_pred, y_true)

class Linear(Layer):
    def __init__(self, in_f, out_f):
        super().__init__()
        self.params['W'] = np.random.randn(in_f, out_f) * np.sqrt(2.0/in_f)
        self.params['b'] = np.zeros(out_f)
    def forward(self, x):
        self.x = x
        return x @ self.params['W'] + self.params['b']
    def backward(self, dout):
        n = self.x.shape[0]
        self.grads['W'] = self.x.T @ dout / n
        self.grads['b'] = np.mean(dout, axis=0)
        return dout @ self.params['W'].T

class ReLU(Layer):
    def forward(self, x):
        self.mask = x > 0
        return np.maximum(0, x)
    def backward(self, dout): return dout * self.mask

class Sigmoid(Layer):
    def forward(self, x):
        self.out = 1/(1+np.exp(-np.clip(x,-500,500)))
        return self.out
    def backward(self, dout): return dout * self.out * (1-self.out)

class MSELoss(Loss):
    def forward(self, y_pred, y_true):
        self.y_pred, self.y_true = y_pred, y_true
        return np.mean((y_pred - y_true)**2)
    def backward(self):
        return 2*(self.y_pred - self.y_true)/self.y_pred.shape[0]

class CrossEntropyLoss(Loss):
    def forward(self, logits, y_true):
        self.y_true = y_true
        n = logits.shape[0]
        exp_l = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        self.probs = exp_l / np.sum(exp_l, axis=1, keepdims=True)
        return -np.mean(np.log(self.probs[np.arange(n), y_true]+1e-10))
    def backward(self):
        n = self.probs.shape[0]
        grad = self.probs.copy()
        grad[np.arange(n), self.y_true] -= 1
        return grad / n

class BinaryCrossEntropyLoss(Loss):
    def forward(self, y_pred, y_true):
        self.y_pred = np.clip(y_pred, 1e-10, 1-1e-10)
        self.y_true = y_true
        return -np.mean(y_true*np.log(self.y_pred) + (1-y_true)*np.log(1-self.y_pred))
    def backward(self):
        return (self.y_pred - self.y_true)/(self.y_pred*(1-self.y_pred)*len(self.y_true))

class Adam:
    def __init__(self, layers, lr=0.001, b1=0.9, b2=0.999, eps=1e-8):
        self.layers, self.lr, self.b1, self.b2, self.eps = layers, lr, b1, b2, eps
        self.m, self.v, self.t = {}, {}, 0
    def step(self):
        self.t += 1
        for i, l in enumerate(self.layers):
            for n, p in l.params.items():
                if n not in l.grads: continue
                k = (i,n)
                g = l.grads[n]
                if k not in self.m: self.m[k], self.v[k] = np.zeros_like(p), np.zeros_like(p)
                self.m[k] = self.b1*self.m[k] + (1-self.b1)*g
                self.v[k] = self.b2*self.v[k] + (1-self.b2)*g**2
                m_h = self.m[k]/(1-self.b1**self.t)
                v_h = self.v[k]/(1-self.b2**self.t)
                l.params[n] -= self.lr * m_h/(np.sqrt(v_h)+self.eps)
    def zero_grad(self):
        for l in self.layers: l.grads = {}

class Sequential:
    def __init__(self, layers): self.layers = layers
    def forward(self, x):
        for l in self.layers: x = l.forward(x)
        return x
    def backward(self, d):
        for l in reversed(self.layers): d = l.backward(d)
        return d
    def __call__(self, x): return self.forward(x)
    def parameters(self): return [l for l in self.layers if l.params]

print("Library geladen!")

---

## Project A: Huizenprijzen Regressie - Oplossing

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data
housing = fetch_california_housing()
X, y = housing.data, housing.target.reshape(-1, 1)

# Preprocessing
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

In [ ]:
# Model
model = Sequential([
    Linear(8, 64),
    ReLU(),
    Linear(64, 32),
    ReLU(),
    Linear(32, 1)
])

criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Training
batch_size = 128
n_epochs = 50
losses = []

for epoch in range(n_epochs):
    idx = np.random.permutation(len(X_train))
    epoch_loss = 0
    n_batches = len(X_train) // batch_size
    
    for batch in range(n_batches):
        start = batch * batch_size
        X_b = X_train[idx[start:start+batch_size]]
        y_b = y_train[idx[start:start+batch_size]]
        
        pred = model(X_b)
        loss = criterion(pred, y_b)
        epoch_loss += loss
        
        optimizer.zero_grad()
        model.backward(criterion.backward())
        optimizer.step()
    
    losses.append(epoch_loss / n_batches)
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}: Loss = {losses[-1]:.4f}")

In [ ]:
# Evaluatie
y_pred = model(X_test)
mse = np.mean((y_pred - y_test)**2)

# R² score
ss_res = np.sum((y_test - y_pred)**2)
ss_tot = np.sum((y_test - np.mean(y_test))**2)
r2 = 1 - ss_res / ss_tot

print(f"Test MSE: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].scatter(y_test, y_pred, alpha=0.5)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[1].set_xlabel('Actual')
axes[1].set_ylabel('Predicted')
axes[1].set_title(f'Predicted vs Actual (R²={r2:.3f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Project B: Iris Classificatie - Oplossing

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target

# Normaliseer
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Split
idx = np.random.permutation(len(X))
X, y = X[idx], y[idx]
X_train, X_test = X[:120], X[120:]
y_train, y_test = y[:120], y[120:]

# Model
model = Sequential([
    Linear(4, 16),
    ReLU(),
    Linear(16, 8),
    ReLU(),
    Linear(8, 3)
])

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Training
for epoch in range(200):
    logits = model(X_train)
    loss = criterion(logits, y_train)
    
    optimizer.zero_grad()
    model.backward(criterion.backward())
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        acc = np.mean(np.argmax(model(X_test), axis=1) == y_test)
        print(f"Epoch {epoch+1}: Loss={loss:.4f}, Test Acc={acc:.4f}")

# Final accuracy
acc = np.mean(np.argmax(model(X_test), axis=1) == y_test)
print(f"\nFinal Test Accuracy: {acc*100:.1f}%")

---

## Project C: Spam Detectie - Oplossing

In [ ]:
# Data
np.random.seed(42)
n_samples = 2000
n_features = 20

X_spam = np.random.exponential(0.5, (n_samples//2, n_features)) + 0.3
X_ham = np.random.exponential(0.3, (n_samples//2, n_features))
X = np.vstack([X_spam, X_ham])
y = np.array([1]*(n_samples//2) + [0]*(n_samples//2)).reshape(-1, 1)

idx = np.random.permutation(n_samples)
X, y = X[idx], y[idx]

X_train, X_test = X[:1600], X[1600:]
y_train, y_test = y[:1600], y[1600:]

# Normalize
X_train = (X_train - X_train.mean(axis=0)) / (X_train.std(axis=0) + 1e-8)
X_test = (X_test - X_train.mean(axis=0)) / (X_train.std(axis=0) + 1e-8)

# Model met Sigmoid output
model = Sequential([
    Linear(20, 32),
    ReLU(),
    Linear(32, 16),
    ReLU(),
    Linear(16, 1),
    Sigmoid()
])

criterion = BinaryCrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training
batch_size = 64
for epoch in range(100):
    idx = np.random.permutation(len(X_train))
    for i in range(0, len(X_train), batch_size):
        X_b = X_train[idx[i:i+batch_size]]
        y_b = y_train[idx[i:i+batch_size]]
        
        pred = model(X_b)
        loss = criterion(pred, y_b)
        
        optimizer.zero_grad()
        model.backward(criterion.backward())
        optimizer.step()
    
    if (epoch+1) % 20 == 0:
        pred_test = (model(X_test) > 0.5).astype(int)
        acc = np.mean(pred_test == y_test)
        print(f"Epoch {epoch+1}: Acc={acc:.4f}")

# Final
pred_test = (model(X_test) > 0.5).astype(int)
acc = np.mean(pred_test == y_test)
print(f"\nFinal Accuracy: {acc*100:.1f}%")

---

## Project D: Digit Sum - Oplossing

In [ ]:
from sklearn.datasets import fetch_openml

# Laad MNIST
print("MNIST laden...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X_mnist, y_mnist = mnist.data / 255.0, mnist.target.astype(int)

# Genereer paren
n_pairs = 20000
idx1 = np.random.randint(0, len(X_mnist), n_pairs)
idx2 = np.random.randint(0, len(X_mnist), n_pairs)

X = np.hstack([X_mnist[idx1], X_mnist[idx2]])
y = y_mnist[idx1] + y_mnist[idx2]

# Split
X_train, X_test = X[:16000], X[16000:]
y_train, y_test = y[:16000], y[16000:]

print(f"Train: {X_train.shape}")

In [ ]:
# Model
model = Sequential([
    Linear(1568, 256),
    ReLU(),
    Linear(256, 128),
    ReLU(),
    Linear(128, 19)  # 0-18
])

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training
batch_size = 128
n_batches = len(X_train) // batch_size

for epoch in range(10):
    idx = np.random.permutation(len(X_train))
    epoch_loss = 0
    
    for batch in range(n_batches):
        start = batch * batch_size
        X_b = X_train[idx[start:start+batch_size]]
        y_b = y_train[idx[start:start+batch_size]]
        
        logits = model(X_b)
        loss = criterion(logits, y_b)
        epoch_loss += loss
        
        optimizer.zero_grad()
        model.backward(criterion.backward())
        optimizer.step()
    
    # Eval
    preds = np.argmax(model(X_test), axis=1)
    acc = np.mean(preds == y_test)
    print(f"Epoch {epoch+1}: Loss={epoch_loss/n_batches:.4f}, Acc={acc:.4f}")

print(f"\nFinal Accuracy: {acc*100:.1f}%")

In [ ]:
# Visualiseer voorbeelden
fig, axes = plt.subplots(2, 5, figsize=(14, 6))

for ax in axes.flatten():
    i = np.random.randint(len(X_test))
    img1 = X_test[i, :784].reshape(28, 28)
    img2 = X_test[i, 784:].reshape(28, 28)
    combined = np.hstack([img1, img2])
    
    pred = np.argmax(model(X_test[i:i+1]))
    true = y_test[i]
    
    ax.imshow(combined, cmap='gray')
    color = 'green' if pred == true else 'red'
    ax.set_title(f'Pred: {pred}, True: {true}', color=color)
    ax.axis('off')

plt.suptitle('Digit Sum Predictions')
plt.tight_layout()
plt.show()

---

## Reflectie Antwoorden

1. **Wiskundige concepten**: Matrixvermenigvuldiging (forward pass), afgeleiden en kettingregel (backward pass), gradient descent (optimalisatie), softmax/cross-entropy (classificatie).

2. **Architectuurkeuze**: Meerdere hidden layers voor voldoende capaciteit, ReLU voor snelle training en geen vanishing gradients, BatchNorm voor stabiele training.

3. **Problemen**: Overfitting → oplossing: dropout, regularisatie. Vanishing gradients → oplossing: ReLU, He initialisatie. Slow convergence → oplossing: Adam optimizer.

4. **Verbeteringen**: Meer experimenten met hyperparameters, learning rate scheduling, data augmentatie, diepere architecturen.

---

**Mathematical Foundations** | Les 12 Oplossingen | IT & Artificial Intelligence

---